In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyRegressor
import pandas as pd
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import (
    ColumnTransformer,
    TransformedTargetRegressor,
    make_column_transformer,
)
import numpy as np

In [2]:
import altair as alt

alt.renderers.enable('html')

RendererRegistry.enable('html')

# Load data

In [3]:
data = pd.read_csv('../data/cleaned_data_v2.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 47 columns):
 #   Column                                                                                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                         --------------  -----  
 0   case number                                                                                                                                                                                                                    696 non-null    object 
 1   Who was the member adjudicating the decision?                                                                                         

In [4]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=123)
X_train, y_train = train_df.drop(columns = ["What was the outcome of the case?"]), train_df["What was the outcome of the case?"]
X_test, y_test = test_df.drop(columns = ["What was the outcome of the case?"]), test_df["What was the outcome of the case?"]
X_train.head()

,case number,Who was the member adjudicating the decision?,What was the location of the landlord tenant board?,Did the decision state the landlord was represented?,Did the decision state the landlord attended the hearing?,Did the decision state the tenant was represented?,Did the decision state the tenant attended the hearing?,Did the decision state the landlord was a not-for-profit landlord (e.g. Toronto Community Housing)?,Did the decision state the tenant was collecting a subsidy?,What was the length of the tenancy,...,L9 present?,N5 present?,N6 present?,N7 present?,N8 present?,T1 present?,T2 present?,T3 present?,T5 present?,T6 present?
195,TSL-90833-17,Roger Rodrigues,Toronto,1,0,0,0,0,0,6.306049,...,0,0,0,0,0,0,0,0,0,0
553,TNL-07861-18,Nancy Morris,Toronto,0,1,0,1,0,0,24.000000,...,0,0,0,0,0,0,0,0,0,0
598,TSL-96267-18,David Mungovan,Toronto,1,0,0,1,0,0,6.306049,...,0,0,0,0,0,0,0,0,0,0
645,TEL-81094-17,Shelby Whittick,Whitby,0,1,0,1,0,0,6.306049,...,0,0,0,0,0,0,0,0,0,0
634,TEL-80073-17,Jim McMaster,Toronto,1,1,0,1,0,0,6.306049,...,0,0,0,0,0,0,1,0,0,0


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 195 to 510
Data columns (total 46 columns):
 #   Column                                                                                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                         --------------  -----  
 0   case number                                                                                                                                                                                                                    556 non-null    object 
 1   Who was the member adjudicating the decision?                                                                                       

In [6]:
y_train.value_counts()

No relief                   352
Postponement of eviction    124
Payment plan                 64
Relief                       12
Conditional Order             4
Name: What was the outcome of the case?, dtype: int64

there is an imbalanced class: 352's No relief and 124's Postponement of eviction and others. Therefore, we should add `class_weight = 'balanced'` in estimator's parameter.

In [7]:
X_train['If the tenant was employed, did the decision state any doubts about the stability of employment e.g. lack of guaranteed hours, contract work, etc.?'].unique()

array([ 0,  1, -1], dtype=int64)

## Preprocess data with pipeline

In [8]:
categorical_features = ['Who was the member adjudicating the decision?',
                        'What was the location of the landlord tenant board?'
                        ]

numeric_features = [
'What was the length of the tenancy',
'What was the monthly rent?',
'rental deposit amount',
'What was the total amount of arrears?',
'Over how many months did the arrears accumulate?',

]
ordinal_features = [
'Was the tenant employed at the time of the hearing?',
'Payment Plan',
'If the tenant was not employed, did the decision state the tenant was receiving any form of government assistance (e.g. OW, childcare benefits, ODSP, OSAP)?',
'If the tenant was employed, did the decision state any doubts about the stability of employment e.g. lack of guaranteed hours, contract work, etc.?'
]
drop_features = ['case number']
ordering_ordinal = [
    [-1,0,1],
    [0, 0.5, 1],
    [ -1, 0, 1],
    [ -1, 0, 1]
]

passthrough_feats = list(
    set(X_train.columns)
    - set(categorical_features)
    - set(numeric_features)
    - set(ordinal_features)
    - set(drop_features)
)

In [9]:
preprocessor = make_column_transformer(
        (OneHotEncoder(handle_unknown="ignore"),categorical_features),
        ("passthrough", passthrough_feats),
        (StandardScaler(),numeric_features),
        (OrdinalEncoder(categories = ordering_ordinal),ordinal_features),
        ("drop",drop_features))
preprocessor

ColumnTransformer(transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Who was the member adjudicating the '
                                  'decision?',
                                  'What was the location of the landlord '
                                  'tenant board?']),
                                ('passthrough', 'passthrough',
                                 ['Did the decision mention any other '
                                  'extenuating circumstances experienced by '
                                  'the tenant leading up to or during the '
                                  'period of the claim (e.g. hospitalizat...
                                 ['Was the tenant employed at the time of the '
                                  'hearing?',
                                  'Payment Plan',
                                  'If the tenant was not employed, did the '
                                  'decision state the tenant was receiving any '
                                  'form of government assistance (e.g. OW, '
                                  'childcare benefits, ODSP, OSAP)?',
                                  'If the tenant was employed, did the '
                                  'decision state any doubts about the '
                                  'stability of employment e.g. lack of '
                                  'guaranteed hours, contract work, etc.?']),
                                ('drop', 'drop', ['case number'])])

## Baseline model

In [10]:
results = {}
scoring_metrics = ["accuracy"]

In [11]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [12]:
from sklearn.dummy import DummyClassifier
results["dummy"] = mean_std_cross_val_scores(
    DummyClassifier(), X_train, y_train, return_train_score=True,scoring = scoring_metrics
)
pd.DataFrame(results).T

c:\Users\chris\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.001 (+/- 0.000),0.000 (+/- 0.000),0.633 (+/- 0.004),0.633 (+/- 0.001)


## Multinomial logistic regression


In [13]:
pipe = make_pipeline(preprocessor, LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000))


results["LR"] = mean_std_cross_val_scores(
    pipe, X_train, y_train, return_train_score=True, scoring=scoring_metrics
)
pd.DataFrame(results).T

c:\Users\chris\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.001 (+/- 0.000),0.000 (+/- 0.000),0.633 (+/- 0.004),0.633 (+/- 0.001)
LR,0.059 (+/- 0.026),0.004 (+/- 0.001),0.736 (+/- 0.055),0.853 (+/- 0.011)


### Hyperparameter tuning
Use randomizedSearchCV to find the best C parameters value.

In [14]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV


lr = LogisticRegression(random_state=123, multi_class='multinomial', solver='lbfgs', max_iter=1000) #class_weight = 'balanced',solver='lbfgs', 
pipe_lr = make_pipeline(preprocessor, lr)

param_dist = { 
    "logisticregression__C": loguniform(1e-3, 1e3),
    "logisticregression__class_weight": ["balanced", None]
}

random_search = RandomizedSearchCV(
            pipe_lr, param_distributions=param_dist, n_iter=20, 
            n_jobs=-1, verbose = 1, scoring= scoring_metrics, 
            refit = 'accuracy', return_train_score=True, random_state=123)

random_search.fit(X_train, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\chris\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['Who '
                                                                                'was '
                                                                                'the '
                                                                                'member '
                                                                                'adjudicating '
                                                                                'the '
                                                                                'decision?',
                                                                                'What '
                                                                                'was '
                                                                                'the '
                                                                                'location '
                                                                                'of '
                                                                                'the '
                                                                                'landlord '
                                                                                'tenant '
                                                                                'board?']),
                                                                              ('passthrough',
                                                                               'passthrough',
                                                                               ['Did '
                                                                                'the '
                                                                                'decision '
                                                                                'mention '
                                                                                'any '
                                                                                'other '
                                                                                'exten...
                                              LogisticRegression(max_iter=1000,
                                                                 multi_class='multinomial',
                                                                 random_state=123))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'logisticregression__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000206F61AB280>,
                                        'logisticregression__class_weight': ['balanced',
                                                                             None]},
                   random_state=123, refit='accuracy', return_train_score=True,
                   scoring=['accuracy'], verbose=1)

In [15]:
random_search.best_params_

{'logisticregression__C': 0.42798984848239946,
 'logisticregression__class_weight': None}

In [16]:
random_search.best_score_

0.7483108108108107

In [17]:
lr_2 = LogisticRegression(random_state=123, C = 0.36913136191432777, solver='lbfgs', multi_class='multinomial', max_iter=1000) 
pipe_lr2 = make_pipeline(preprocessor, lr_2)

results["LR_noweighted_Best"] = mean_std_cross_val_scores(
    pipe_lr2, X_train, y_train, return_train_score=True, scoring=scoring_metrics
)
pd.DataFrame(results).T

c:\Users\chris\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.001 (+/- 0.000),0.000 (+/- 0.000),0.633 (+/- 0.004),0.633 (+/- 0.001)
LR,0.059 (+/- 0.026),0.004 (+/- 0.001),0.736 (+/- 0.055),0.853 (+/- 0.011)
LR_noweighted_Best,0.032 (+/- 0.004),0.005 (+/- 0.001),0.746 (+/- 0.039),0.825 (+/- 0.012)


## set class_weight = 'balanced'

In [18]:
lr3 = LogisticRegression(random_state=123, class_weight = 'balanced',solver='lbfgs', multi_class='multinomial', max_iter=1000)
pipe_lr3 = make_pipeline(preprocessor, lr3)

param_dist = { 
    "logisticregression__C": loguniform(1e-3, 1e3)
}

random_search = RandomizedSearchCV(
            pipe_lr3, param_distributions=param_dist, n_iter=20, 
            n_jobs=-1, verbose = 1, scoring= scoring_metrics, 
            refit = 'accuracy', return_train_score=True, random_state=123)

random_search.fit(X_train, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\chris\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['Who '
                                                                                'was '
                                                                                'the '
                                                                                'member '
                                                                                'adjudicating '
                                                                                'the '
                                                                                'decision?',
                                                                                'What '
                                                                                'was '
                                                                                'the '
                                                                                'location '
                                                                                'of '
                                                                                'the '
                                                                                'landlord '
                                                                                'tenant '
                                                                                'board?']),
                                                                              ('passthrough',
                                                                               'passthrough',
                                                                               ['Did '
                                                                                'the '
                                                                                'decision '
                                                                                'mention '
                                                                                'any '
                                                                                'other '
                                                                                'exten...
                                             ('logisticregression',
                                              LogisticRegression(class_weight='balanced',
                                                                 max_iter=1000,
                                                                 multi_class='multinomial',
                                                                 random_state=123))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'logisticregression__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000206F6A34CA0>},
                   random_state=123, refit='accuracy', return_train_score=True,
                   scoring=['accuracy'], verbose=1)

In [19]:
print(random_search.best_params_)
print(random_search.best_score_)

{'logisticregression__C': 1.5463515822289584}
0.6512065637065637


In [20]:
lr_3 = LogisticRegression(random_state=123, C = 1.5463515822289584,class_weight = 'balanced', solver='lbfgs', multi_class='multinomial') 
pipe_lr3 = make_pipeline(preprocessor, lr_3)

results["LR_weighted_Best"] = mean_std_cross_val_scores(
    pipe_lr3, X_train, y_train, return_train_score=True, scoring=scoring_metrics
)
pd.DataFrame(results).T

c:\Users\chris\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\chris\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\chris\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org

,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.001 (+/- 0.000),0.000 (+/- 0.000),0.633 (+/- 0.004),0.633 (+/- 0.001)
LR,0.059 (+/- 0.026),0.004 (+/- 0.001),0.736 (+/- 0.055),0.853 (+/- 0.011)
LR_noweighted_Best,0.032 (+/- 0.004),0.005 (+/- 0.001),0.746 (+/- 0.039),0.825 (+/- 0.012)
LR_weighted_Best,0.031 (+/- 0.004),0.005 (+/- 0.001),0.651 (+/- 0.069),0.812 (+/- 0.023)


## Feature Importance

In [21]:
pipe_lr2.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Who was the member '
                                                   'adjudicating the decision?',
                                                   'What was the location of '
                                                   'the landlord tenant '
                                                   'board?']),
                                                 ('passthrough', 'passthrough',
                                                  ['Did the decision mention '
                                                   'any other extenuating '
                                                   'circumstances experienced '
                                                   'by the tenant leading up '
                                                   'to or d...
                                                   'government assistance '
                                                   '(e.g. OW, childcare '
                                                   'benefits, ODSP, OSAP)?',
                                                   'If the tenant was '
                                                   'employed, did the decision '
                                                   'state any doubts about the '
                                                   'stability of employment '
                                                   'e.g. lack of guaranteed '
                                                   'hours, contract work, '
                                                   'etc.?']),
                                                 ('drop', 'drop',
                                                  ['case number'])])),
                ('logisticregression',
                 LogisticRegression(C=0.36913136191432777, max_iter=1000,
                                    multi_class='multinomial',
                                    random_state=123))])

In [22]:
column_names = (
    pipe_lr2.named_steps['columntransformer'].named_transformers_['onehotencoder'].get_feature_names_out(categorical_features).tolist()
    + passthrough_feats    
    + numeric_features
    + ordinal_features
)
len(column_names)

126

In [23]:
pipe_lr2.named_steps['logisticregression'].coef_.shape

(5, 126)

The coefficients have an array of (5, 166) because we have a multiclass cases here (5 classes). Thus, an one-vs-rest strategy is applied. Sklearn creates 5 classifiers, that is why we have (5, 166) coefficient arrays. Then, for the feature selection we need to use squared coefficient approach. [reference paper](https://www.nature.com/articles/s41374-021-00662-x)

In [24]:
def squared_coef(coefs):
    """ 
    For each feature,  calculated the squared-coefficient of each
    feature by adding together the squared values of the feature's coefficient
    for each class.
    """
    squared_coef = np.square(coefs)
    results = np.sum(squared_coef, axis=0)
    return results


In [25]:
squared_coef(pipe_lr2.named_steps['logisticregression'].coef_).shape

(126,)

In [26]:
coefs= (
    pd.DataFrame(squared_coef(pipe_lr2.named_steps['logisticregression'].coef_), column_names, columns=["coef"]) # check first classifier
    .query("coef != 0")
    .sort_values("coef",ascending=False)
    .reset_index()
    .rename(columns={"index": "variable"})
)
# coefs.style.background_gradient('PuOr')

In [27]:
coefs

,variable,coef
0,Payment Plan,3.519561
1,Did the decision state the tenant attended the...,1.921413
2,Did the member find the tenant had sufficient ...,1.588751
3,Did the decisions state postponement would res...,1.551946
4,Did the decision mention the tenant’s difficul...,1.394449
...,...,...
121,Who was the member adjudicating the decision?_...,0.001051
122,Who was the member adjudicating the decision?_...,0.000485
123,What was the location of the landlord tenant b...,0.000469
124,T1 present?,0.000365


In [28]:


alt.Chart(
    coefs,
    title=pipe_lr.named_steps['logisticregression'].__str__()[:-2] + " Coefficients",
).mark_bar().encode(y=alt.Y("variable", sort="-x"), x="coef")

alt.Chart(...)

## Conclusion:

It seems like if we add the `class_weight = 'balanced'` and default LR, the classfier is overfit with the training data. Thus, we will set the `class_weight = None`

That is `pipe_lr2`

In [29]:
pipe_lr2.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Who was the member '
                                                   'adjudicating the decision?',
                                                   'What was the location of '
                                                   'the landlord tenant '
                                                   'board?']),
                                                 ('passthrough', 'passthrough',
                                                  ['Did the decision mention '
                                                   'any other extenuating '
                                                   'circumstances experienced '
                                                   'by the tenant leading up '
                                                   'to or d...
                                                   'government assistance '
                                                   '(e.g. OW, childcare '
                                                   'benefits, ODSP, OSAP)?',
                                                   'If the tenant was '
                                                   'employed, did the decision '
                                                   'state any doubts about the '
                                                   'stability of employment '
                                                   'e.g. lack of guaranteed '
                                                   'hours, contract work, '
                                                   'etc.?']),
                                                 ('drop', 'drop',
                                                  ['case number'])])),
                ('logisticregression',
                 LogisticRegression(C=0.36913136191432777, max_iter=1000,
                                    multi_class='multinomial',
                                    random_state=123))])

In [30]:
pipe_lr2.score(X_test, y_test)

0.7428571428571429

In [31]:
pd.DataFrame(results).T

,fit_time,score_time,test_accuracy,train_accuracy
dummy,0.001 (+/- 0.000),0.000 (+/- 0.000),0.633 (+/- 0.004),0.633 (+/- 0.001)
LR,0.059 (+/- 0.026),0.004 (+/- 0.001),0.736 (+/- 0.055),0.853 (+/- 0.011)
LR_noweighted_Best,0.032 (+/- 0.004),0.005 (+/- 0.001),0.746 (+/- 0.039),0.825 (+/- 0.012)
LR_weighted_Best,0.031 (+/- 0.004),0.005 (+/- 0.001),0.651 (+/- 0.069),0.812 (+/- 0.023)
